## ngl::Mat4

The ```ngl::Mat4``` class is a simple 4x4 matrix of floating point values.

Internally the data is stored in a [union](https://en.cppreference.com/w/cpp/language/union) like this

```c++
union
{
    Real m_m[4][4];
    std::array<Real,16> m_openGL=
    {{
        1.0f,0.0f,0.0f,0.0f,
        0.0f,1.0f,0.0f,0.0f,
        0.0f,0.0f,1.0f,0.0f,
        0.0f,0.0f,0.0f,1.0f
    }};
      struct
      {
          Real m_00; 
          Real m_01; 
          Real m_02; 
          Real m_03; 
          Real m_10; 
          Real m_11; 
          Real m_12; 
          Real m_13; 
          Real m_20; 
          Real m_21; 
          Real m_22; 
          Real m_23; 
          Real m_30; 
          Real m_31; 
          Real m_32; 
          Real m_33;  
      };
};

```

In [1]:
// You may need to modify these paths to suit
#pragma cling add_library_path("$HOME/NGL/lib")
#ifdef __APPLE__
    #pragma cling load("$HOME/NGL/lib/libNGL.dylib")
#else
    #pragma cling load("$HOME/NGL/lib/libNGL.so")
#endif
#pragma cling add_include_path("$HOME/NGL/include")
#pragma cling add_include_path("$HOME/NGL/gl3w")

In [2]:
#include <ngl/Vec4.h>
#include <ngl/Mat4.h>
#include <ngl/NGLStream.h> // for printing
#include <iostream>


To construct an ```ngl::Mat4``` we can do the following

In [3]:
{
    ngl::Mat4 defaultCtor;
    ngl::Mat4 user(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16);
    ngl::Mat4 scaleBy10(10.0f);
    std::cout<<"default \n"<<defaultCtor<<'\n';
    std::cout<<"user \n"<<user<<'\n';
    std::cout<<"scaleBy10 \n"<<scaleBy10<<'\n';
}

default 
[1,0,0,0]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]

user 
[1,5,9,13]
[2,6,10,14]
[3,7,11,15]
[4,8,12,16]

scaleBy10 
[10,0,0,0]
[0,10,0,0]
[0,0,10,0]
[0,0,0,1]



All of the attributes in a Mat4 are public and can be accessed in the following ways

In [4]:
{
    ngl::Mat4 m;
    m.m_00=99.0f;
    m.m_m[0][1]=8.0f;
    m.m_10=-2.0f;
    m.m_openGL[3]=100.0f;
    m.m_22=8.27f;
    m.m_33=99.27f;
    
    std::cout<<m<<'\n';
    
}

[99,-2,0,0]
[8,1,0,0]
[0,0,8.27,0]
[100,0,0,99.27]



We can zero the matrix using the null() method.

In [5]:
{
    ngl::Mat4 a;
    std::cout<<a<<'\n';
    a.null();
    std::cout<<"\nafter null\n"<<a<<'\n';
}

[1,0,0,0]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]


after null
[0,0,0,0]
[0,0,0,0]
[0,0,0,0]
[0,0,0,0]



We can reset a matrix to the [identity matrix](https://en.wikipedia.org/wiki/Identity_matrix) using the .identity() method. 

In [6]:
{
    auto m=ngl::Mat4{1.0f,2.0f,3.0f,4.0f,5.0f,6.0f,7.0f,8.0f,
                     9.0f,10.0f,11.0f,12.0f,13.0f,14.0f,15.0f,16.0f};
    std::cout<<m<<'\n';
    m.identity();
    std::cout<<"\nafter identity\n"<<m<<'\n';
}

[1,5,9,13]
[2,6,10,14]
[3,7,11,15]
[4,8,12,16]


after identity
[1,0,0,0]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]



### Transformations

```ngl::Mat4``` has a number of transformation functions

In [7]:
{
   ngl::Mat4 m{1.0f,2.0f,3.0f,4.0f,5.0f,6.0f,7.0f,8.0f,
               9.0f,10.0f,11.0f,12.0f,13.0f,14.0f,15.0f,16.0f};
   std::cout<<m<<'\n';
   m.transpose();
   std::cout<<"transpose \n" <<m<<'\n';
}

[1,5,9,13]
[2,6,10,14]
[3,7,11,15]
[4,8,12,16]

transpose 
[1,2,3,4]
[5,6,7,8]
[9,10,11,12]
[13,14,15,16]



We can set the matrix to be a scale matrix using the scale method;

In [8]:
{
    ngl::Mat4 scale;
    scale.scale(0.5f,0.3f,0.2f);
    std::cout<<scale<<'\n';
}

[0.5,0,0,0]
[0,0.3,0,0]
[0,0,0.2,0]
[0,0,0,1]



And create a simple rotation matrix by calling the rotate method passing in the rotation in degrees.

$$
R_x(\beta)=\begin{pmatrix}
1 & 0 & 0 & 0 \\\
0 & \cos(\beta) & -\sin(\beta) & 0 \\\
0 & \sin(\beta) & \cos(\beta) & 0 \\\
0 & 0 & 0 & 1 
\end{pmatrix}
$$ 

```c++
void Mat4::rotateX( const Real _deg) noexcept
{
  Real beta=radians(_deg);
  Real sr = sinf( beta );
  Real cr = cosf( beta );
  m_11 =  cr;
  m_21 = -sr;
  m_12 =  sr;
  m_22 =  cr;
}
```

$$
R_y(\beta)=\begin{pmatrix}
\cos(\beta) & 0 & \sin(\beta) & 0 \\\
0 & 1 & 0 & 0  \\\
-\sin(\beta) & 0 & \cos(\beta) & 0 \\\
0 & 0 & 0 & 1 
\end{pmatrix}
$$ 

```c++
void Mat4::rotateY( const Real _deg ) noexcept
{
  Real beta=radians(_deg);
  Real sr = sinf( beta );
  Real cr = cosf( beta );
  m_00 =  cr;
  m_20 =  sr;
  m_02 = -sr;
  m_22 =  cr;
}
```

$$
R_z(\beta)=\begin{pmatrix}
\cos(\beta) & -\sin(\beta) & 0 & 0 \\\
\sin(\beta) &  \cos(\beta) & 0 & 0 \\\
0 & 0 & 1 & 0 \\\
0 & 0 & 0 & 1 
\end{pmatrix}
$$ 

```c++
void Mat4::rotateZ(const Real _deg ) noexcept
{
  Real beta=radians(_deg);
  Real sr = sinf( beta );
  Real cr = cosf( beta );
  m_00 =  cr;
  m_10 = -sr;
  m_01 =  sr;
  m_11 =  cr;
}
```

In [9]:
{
    auto tx=ngl::Mat4();
    tx.rotateX(35.0f);
    std::cout<<tx<<'\n';
    tx.rotateY(25.0f);
    std::cout<<tx<<'\n';
    tx.rotateZ(85.0f);
    std::cout<<tx<<'\n';
    
}

[1,0,0,0]
[0,0.819152,-0.573577,0]
[0,0.573577,0.819152,0]
[0,0,0,1]

[0.906308,0,0.422618,0]
[0,0.819152,-0.573577,0]
[-0.422618,0.573577,0.906308,0]
[0,0,0,1]

[0.0871558,-0.996195,0.422618,0]
[0.996195,0.0871558,-0.573577,0]
[-0.422618,0.573577,0.906308,0]
[0,0,0,1]



Typically we will combine a series of rolls and scales.

In [24]:
{
    ngl::Mat4 rx,ry,rz,scale,translate;
    rx.rotateX(25.0f);
    ry.rotateY(12.0f);
    rz.rotateZ(5.0f);
    scale.scale(0.2f,0.2f,0.2f);
    translate.translate(0.2,1.0f,2.0f);
    auto tx=scale*rz*ry*rx;
    tx.m_m[3][0]=translate.m_30;
    tx.m_m[3][1]=translate.m_31;
    tx.m_m[3][2]=translate.m_32; // easier to set tx manually
    std::cout<<tx<<'\n';
}

[0.194885,0.0017086,0.0449097,0.2]
[0.0170502,0.182103,-0.0809174,1]
[-0.0415823,0.0826766,0.177301,2]
[0,0,0,1]



We can also do [Euler rotations](https://en.wikipedia.org/wiki/Euler_angles) around an axis. 

In [11]:
{
    ngl::Mat4 rotX;
    rotX.euler(45.0f,1,0,0);
    std::cout<<rotX<<'\n';
    ngl::Mat4 rotY;
    rotY.euler(90.0f,0,1,0);
    std::cout<<rotY<<'\n';
    ngl::Mat4 rotZ;
    rotZ.euler(20.0f,0,0,1);
    std::cout<<rotZ<<'\n';

}

[1,0,0,0]
[0,0.707107,-0.707107,0]
[0,0.707107,0.707107,0]
[0,0,0,1]

[-4.37114e-08,0,1,0]
[0,1,0,0]
[-1,0,-4.37114e-08,0]
[0,0,0,1]

[0.939693,-0.34202,0,0]
[0.34202,0.939693,0,0]
[0,0,1,0]
[0,0,0,1]



## Matrix math operations

Standard matrix math operations are available via operator overloading.

In [12]:
{
    ngl::Mat4 a(1.0f,2.0f,3.0f,4.0f,5.0f,6.0f,7.0f,8.0f,9.0f,
                10.0f,11.0f,12.0f,13.0f,14.0f,15.0f,16.0f);
    ngl::Mat4 b(0.2f);
    auto result=a*b;
    std::cout<<result<<'\n';  
}

[0.2,1,1.8,13]
[0.4,1.2,2,14]
[0.6,1.4,2.2,15]
[0.8,1.6,2.4,16]



In [13]:
{
    ngl::Mat4 a{1.0f,2.0f,3.0f,4.0f,5.0f,6.0f,7.0f,8.0f,9.0f,
                10.0f,11.0f,12.0f,13.0f,14.0f,15.0f,16.0f};
    ngl::Mat4 b(0.2f);
    a*=b;
    std::cout<<a<<'\n';
}

[0.2,1,1.8,13]
[0.4,1.2,2,14]
[0.6,1.4,2.2,15]
[0.8,1.6,2.4,16]



In [14]:
{
    ngl::Mat4 a{1.0f,2.0f,3.0f,4.0f,5.0f,6.0f,7.0f,8.0f,9.0f,
                10.0f,11.0f,12.0f,13.0f,14.0f,15.0f,16.0f};
    ngl::Mat4 b{17.0f,18.0f,19.0f,20.0f,21.0f,22.0f,23.0f,24.0f,
                25.0f,26.0f,27.0f,28.0f,29.0f,30.0f,31.0f,32.0f};
    auto result=a+b;
    std::cout<<result<<'\n';
}

[18,26,34,42]
[20,28,36,44]
[22,30,38,46]
[24,32,40,48]



In [15]:
{
    ngl::Mat4 a{1.0f,2.0f,3.0f,4.0f,5.0f,6.0f,7.0f,8.0f,9.0f,
                10.0f,11.0f,12.0f,13.0f,14.0f,15.0f,16.0f};
    ngl::Mat4 b{17.0f,18.0f,19.0f,20.0f,21.0f,22.0f,23.0f,24.0f,
                25.0f,26.0f,27.0f,28.0f,29.0f,30.0f,31.0f,32.0f};
    a+=b;
    std::cout<<a<<'\n';
}

[18,26,34,42]
[20,28,36,44]
[22,30,38,46]
[24,32,40,48]



We can multiply each matrix element by a scalar

In [16]:
{
    ngl::Mat4 a{1.0f,2.0f,3.0f,4.0f,5.0f,6.0f,7.0f,8.0f,9.0f,
                10.0f,11.0f,12.0f,13.0f,14.0f,15.0f,16.0f};
    
    auto b=a*0.5f;
    std::cout<<b<<'\n';
    a*=0.2f;
    std::cout<<a<<'\n';
}

[0.5,2.5,4.5,6.5]
[1,3,5,7]
[1.5,3.5,5.5,7.5]
[2,4,6,8]

[0.2,1,1.8,2.6]
[0.4,1.2,2,2.8]
[0.6,1.4,2.2,3]
[0.8,1.6,2.4,3.2]



We can also multiply and ```ngl::Vec4``` by a ```ngl::Mat4```

In [17]:
{
    ngl::Vec4 p1{0.0f,1.0f,2.0f};
    ngl::Mat4 rx,ry,rz,scale;
    rx.rotateX(25.0f);
    ry.rotateY(12.0f);
    rz.rotateZ(5.0f);
    scale.scale(0.2f,0.2f,0.2f);
    auto tx=scale*rz*ry*rx;
    std::cout<<tx<<'\n';

    std::cout<<"p1 * tx = "<<tx*p1<<'\n';
    std::cout<<"p1 * tx ="<<p1*tx<<'\n';
}

[0.194885,0.0017086,0.0449097,0]
[0.0170502,0.182103,-0.0809174,0]
[-0.0415823,0.0826766,0.177301,0]
[0,0,0,1]

p1 * tx = [0.091528,0.0202686,0.437278,1]
p1 * tx =[-0.0661144,0.347457,0.273684,1]


The [determinant](https://en.wikipedia.org/wiki/Determinant) of the matrix can be calculated with the determinant() method

In [18]:
{
    ngl::Mat4 test(1,0,0,0,0,2,2,0,0,-0.5,2,0,0,0,0,1);
    float det=test.determinant(); 
    std::cout<<test<<'\n'<<det<<'\n';
}

[1,0,0,0]
[0,2,-0.5,0]
[0,2,2,0]
[0,0,0,1]

5


The [inverse](http://mathworld.wolfram.com/MatrixInverse.html) of the matrix

In [19]:
{
    ngl::Mat4 test(1,0,0,0,0,2,2,0,0,-0.5,2,0,0,0,0,1);
    test=test.inverse();
    std::cout<<test<<'\n';
}

[1,0,0,0]
[0,0.4,0.1,0]
[0,-0.4,0.4,0]
[0,0,0,1]



The [adjacent](https://en.wikipedia.org/wiki/Adjugate_matrix) of the matrix

In [20]:
{
  ngl::Mat4 test(1,0,0,0,0,2,2,0,0,-0.5,2,0,0,0,0,1);
  test=test.adjacent();
  std::cout<<test<<'\n';
}

[5,0,0,0]
[0,2,-2,0]
[0,0.5,2,0]
[0,0,0,5]



In [21]:
{
    ngl::Mat4 t1;
    ngl::Mat4 t2;
    t1.rotateX(45.0f);
    t2.rotateY(35.0f);
    ngl::Mat4 test=t1.adjacent(t2);
    std::cout<<test<<'\n';
}

[0.819152,0,0.573577,0]
[0,1,0,0]
[-0.573577,0,0.819152,0]
[0,0,0,1]

